In [111]:
import numpy as np
import pandas as pd
import torch 
import nltk
import re


In [90]:
movies_train = []
with open('/Users/parthsdambhare/Documents/Enc/Movie classifier/Genre Classification Dataset/train_data.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(':::') 
        title = parts[1]
        plot = parts[3]
        genre = parts[2] 
        movies_train.append([title , plot, genre])
        
movies_test = []
with open('/Users/parthsdambhare/Documents/Enc/Movie classifier/Genre Classification Dataset/test_data_solution.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(':::') 
        title = parts[1]
        plot = parts[3]
        genre = parts[2] 
        movies_test.append([title, plot, genre])        

df_train = pd.DataFrame(movies_train, columns=['Title', 'Plot', 'Genre'])
print(df_train.head())

df_test = pd.DataFrame(movies_test, columns=['Title', 'Plot', 'Genre'])
print(df_test.head())

                           Title  \
0          Edgar's Lunch (1998)    
1      La guerra de papá (1977)    
2   Off the Beaten Track (2010)    
3        Meu Amigo Hindu (2015)    
4             Er nu zhai (1955)    

                                                Plot          Genre  
0   L.R. Brane loves his life - his car, his apar...      thriller   
1   Spain, March 1964: Quico is a very naughty ch...        comedy   
2   One year in the life of Albin and his family ...   documentary   
3   His father has died, he hasn't spoken with hi...         drama   
4   Before he was known internationally as a mart...         drama   
                           Title  \
0          Edgar's Lunch (1998)    
1      La guerra de papá (1977)    
2   Off the Beaten Track (2010)    
3        Meu Amigo Hindu (2015)    
4             Er nu zhai (1955)    

                                                Plot          Genre  
0   L.R. Brane loves his life - his car, his apar...      thriller   
1   Sp

In [91]:
print(df_train.info())  
df_train.dropna(inplace=True)  

print(df_test.info())  
df_test.dropna(inplace=True)  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   54214 non-null  object
 1   Plot    54214 non-null  object
 2   Genre   54214 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54200 entries, 0 to 54199
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   54200 non-null  object
 1   Plot    54200 non-null  object
 2   Genre   54200 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB
None


In [92]:
from sklearn.model_selection import train_test_split

X_train = df_train[['Title', 'Plot']].values 
X_test = df_test[['Title', 'Plot']].values
y_train = df_train[['Genre']].values
y_test = df_test[['Genre']].values


In [93]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/parthsdambhare/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/parthsdambhare/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/parthsdambhare/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [99]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # used for removing special characters
    text = text.lower()  # used for lowercasing the text
    tokens = word_tokenize(text)  # Tokenize the text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Remove stopwords and lemmatize
    return ' '.join(tokens)

# Apply preprocessing to the 'Plot' column
df_train['processed_plot'] = df_train['Plot'].apply(preprocess_text)
# If you have a test DataFrame as well
df_test['processed_plot'] = df_test['Plot'].apply(preprocess_text)  # Replace 'Plot' with your actual column name


In [100]:
df_train.columns = df_train.columns.str.strip()
df_test.columns = df_test.columns.str.strip()


In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)  # You can adjust the max features
X = tfidf.fit_transform(df_train['processed_plot']).toarray()  # Convert text to vectors


# Step 1: Fit and transform the training data (raw text)
X_train_vectorized = tfidf.fit_transform(df_train['processed_plot'])  # Use raw text here

# Step 2: Transform the test data (raw text)
X_test_vectorized = tfidf.transform(df_test['processed_plot'])  # Use raw text here


In [106]:
from sklearn.preprocessing import MultiLabelBinarizer

# Assuming genres are comma-separated in the text
#f_train['Genre'] = df_train['Genre'].apply(lambda x: x.split(','))  # Split genres into lists

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df_train['Genre'])  # Binarize the genres


In [96]:
# Initialize TF-IDF vectorizer
#tfidf = TfidfVectorizer(max_features=5000)  # You can adjust max_features as per your need

# Fit and transform the training data (this converts text to numeric form)
#X_train_vectorized = tfidf.transform(X_train)

# Transform the test data using the same TF-IDF fitting
#X_test_vectorized = tfidf.transform(X_test)

# Now X_train_vectorized and X_test_vectorized can be used for training and testing

In [107]:
'''from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=mlb.classes_))'''


'from sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import accuracy_score, classification_report\n\nmodel = LogisticRegression(max_iter=200)\nmodel.fit(X_train, y_train)\n\ny_pred = model.predict(X_test)\n\nprint("Accuracy:", accuracy_score(y_test, y_pred))\nprint("Classification Report:\n", classification_report(y_test, y_pred, target_names=mlb.classes_))'

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted')}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')}")


from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}
grid_search = GridSearchCV(LogisticRegression(max_iter=200), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print(f"Best Parameters: {grid_search.best_params_}")


from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)


import joblib

# Save the trained model
joblib.dump(model, 'movie_genre_classifier.pkl')

# Later, load the model
model = joblib.load('movie_genre_classifier.pkl')


In [119]:
from torch.utils.data import Dataset

class MovieGenreDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
    # Ensure data is numeric and correctly formatted
        data = np.array(self.data[idx], dtype=np.float32)
        label = int(self.labels[idx])
        return data, label

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True,
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)  # Use float for multi-label
        }


In [112]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = 'bert-base-uncased'  # You can choose another model as needed
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=y.shape[1])  # Number of genres


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [113]:
from torch.utils.data import DataLoader

MAX_LEN = 128
BATCH_SIZE = 16

train_dataset = MovieGenreDataset(X_train.tolist(), y_train, tokenizer, MAX_LEN)
test_dataset = MovieGenreDataset(X_test.tolist(), y_test, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

# Assuming you have a list of texts
texts = ["This is an example.", "More text data here."]
vectorizer = TfidfVectorizer()

# Fit the vectorizer and transform the texts into TF-IDF format
X_train = vectorizer.fit_transform(texts).toarray()
X_test = vectorizer.fit_transform(texts).toarray()

y_train = vectorizer.fit_transform(texts).toarray()
y_test = vectorizer.fit_transform(texts).toarray()

# Convert to PyTorch tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [120]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training Loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')


/Users/parthsdambhare/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.